In [1]:
import numpy as np
import pandas as pd

In [2]:
file_path = 'train_sample.csv'
data = pd.read_csv(file_path, nrows=300000)
data.head()

Unnamed: 0                    id  click      hour    C1  banner_pos  \
0           0   1000009418151094273      0  14102100  1005           0   
1           1  10000169349117863715      0  14102100  1005           0   
2           2  10000371904215119486      0  14102100  1005           0   
3           3  10000640724480838376      0  14102100  1005           0   
4           4  10000679056417042096      0  14102100  1005           1   

    site_id site_domain site_category    app_id ...  device_type  \
0  1fbe01fe    f3845767      28905ebd  ecad2386 ...            1   
1  1fbe01fe    f3845767      28905ebd  ecad2386 ...            1   
2  1fbe01fe    f3845767      28905ebd  ecad2386 ...            1   
3  1fbe01fe    f3845767      28905ebd  ecad2386 ...            1   
4  fe8cc448    9166c161      0569f928  ecad2386 ...            1   

  device_conn_type    C14  C15 C16   C17  C18  C19     C20  C21  
0                2  15706  320  50  1722    0   35      -1   79  
1                0  15704  320  50  1722    0   35  100084   79  
2                0  15704  320  50  1722    0   35  100084   79  
3                0  15706  320  50  1722    0   35  100084   79  
4                0  18993  320  50  2161    0   35      -1  157  

[5 rows x 25 columns]

In [3]:
print(data.columns)
print(data['click'].value_counts())

Index(['Unnamed: 0', 'id', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')
0    8294
1    1706
Name: click, dtype: int64


In [4]:
data = data.drop(['id'], axis=1)
print(data.columns)

Index(['Unnamed: 0', 'click', 'hour', 'C1', 'banner_pos', 'site_id',
       'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21'],
      dtype='object')


In [5]:
for fea in data.drop('click', axis=1).columns:
    data[fea] = pd.factorize(data[fea])[0]

from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)

In [7]:
import lightgbm as lgb

In [8]:
clf = lgb.LGBMClassifier(is_unbalanced=False, slient=False)
cal = list(train.drop('click', axis=1).columns)
print(cal)

['Unnamed: 0', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']


In [10]:
train['unnamed'] = train.pop('Unnamed: 0')
cal = list(train.drop('click', axis=1).columns)
print(cal)
clf.fit(train.drop('click', axis=1), train['click'], categorical_feature=cal, verbose=5)
predict = clf.predict_proba(test.drop('click', axis=1))[:, 1]

def celoss(target, predict):
    target = np.array(target)
    predict = np.array(predict)
    return -(target*np.log(predict)+(1-target)*np.log(1-predict)).mean()

print(celoss(test['click'], predict))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/lightgbm/basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['C1', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'app_category', 'app_domain', 'app_id', 'banner_pos', 'device_conn_type', 'device_id', 'device_ip', 'device_model', 'device_type', 'hour', 'site_category', 'site_domain', 'site_id', 'unnamed']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


['hour', 'C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'unnamed']
0.551570857693822
